# The Application layer

As every person who have used the internet, you are probably 
aware that we interact with remote devices via the internet using applications.

These remote devices may be from our family and friends. Or they can be from 
complete strangers that rent their servers for another strangers that want
a place to host their own applications. These infrastructure providers are
currently called cloud service providers (e.g. AWS, Azure, etc), and the 
renters are known as tenant customers.

Let's begin by visualing a basic network setup.

![](https://camo.githubusercontent.com/089f931f4b5a222a610402457c63606f8b00881c3ef2c058202f7463b0353025/68747470733a2f2f6d65726d6169642e696e6b2f696d672f70616b6f3a654e714e6b5646727779415178372d4b33484d36744e415847594f7548625451684c44745a656765624c7931736d6d434d512d6a394c74505a32414a665a6b48705f3776725f7934753044546167514f4a362d364d7a6b3853306669366f646a467170597055536b6a6444335845784c4734394e4d4b306a6a36395f3672366d596c397a77756764537a4635554b3433564d51306b657264477855786a5249364c56324b477741324172425f414c41707748494f7747344232427767365f6d363261327236756b6756715138646e31426c7354326f7a4f523379385744364e6c356b39362d68674b734f69744d6a7032393549734573495a4c557267386169565f35516733545836314244616c325f584141392d77414b475471754157364e6943797a77445f585652785731436130763837682d7033623941577a7266464d3f747970653d706e67)

Now that we have decided on our network topology, we can implement it in ns-3.

In [1]:
from ns import ns

nodes = ns.network.NodeContainer()
nodes.Create(2)

pointToPoint = ns.point_to_point.PointToPointHelper()
pointToPoint.SetDeviceAttribute("DataRate", ns.core.StringValue("5Mbps"))
pointToPoint.SetChannelAttribute("Delay", ns.core.StringValue("2ms"))

devices = pointToPoint.Install(nodes)
pointToPoint.EnablePcapAll("./application", True)

stack = ns.internet.InternetStackHelper()
stack.Install(nodes)

address = ns.internet.Ipv4AddressHelper()
address.SetBase(ns.network.Ipv4Address("10.1.1.0"),
                ns.network.Ipv4Mask("255.255.255.0"))

interfaces = address.Assign(devices)

In the setup above we have two devices, called Node 0 and Node 1.

These network nodes, as they are usually called, can contain multiple network interfaces. 

Think of your phone: it has 4G, 5G, Wi-Fi, Bluetooth, NFC, plus the USB port used for charging and data. 

Some may have additional satellite communication capabilities, others walk-talkie like capabilities, maybe even a infrared interface, or a lightning port for Apple afficionados.

All of these network interfaces are also called network devices (or NetDevice in ns-3). 

Each of them will contain at least the Medium Access Control (MAC) and Physical Layers (PHY), which are used to exchange messages via a link, be it based on radiofrequency, acoustics, light, etc. 

We are not going to focus on network devices now. But focus on the top.

Notice we have an IP layer. IP stands for Internet Procotol, which is the most commonly used network layer protocol because it is the one used to communicate in the internet. Operating systems or standalone network stacks can use many other network layer protocols, such as [those available on Linux](https://github.com/torvalds/linux/blob/master/net/socket.c#L170-L217).

The IP layer is responsible for routing our application packages to their destination.

But we currently don't have any application, so let's add one.

[![](https://mermaid.ink/img/pako:eNqVk1FrgzAUhf9KyLMO47aXsBZcV6h0FZntw9A9pOZ2lWmUGAej9L8vadywa0uZQtR7DudeP80O5zUHTPG7ZM0WLR8zgfTRdmtbiLTqodRckPdmxSPDKtHyqgWJ2oblMLAEceylK95M8209KQsQiiLt-wQZNiPi3RB9-k5fimupRsS_vRsEgOBnGs5NwzlIAeVJy-VLECW2K0WSCV5XqNHJA0uo5TCmqB-ADKRFMPFSvQxK8ezVS_VyaSrzjg-um9T5ByjXHfcDWMPBeYYn6XmS8zzJZZ7kh2dygEZRWbQKxH_gkSvwiIVnqKE_kaEWB-j8Y3TkFB25go6coiO_6KzLPk5mQRRNn9N7tFg3rYN8VLV9rvlCOmbcW478pm7GwA6uQFas4PpH3xlLhtUWKsgw1bccNqwrVYYzsddW1qk6-RI5pkp24OCu4UzBU8E0wgrTDStbXQVeqFou7OY57KH9N3Ti7rk?type=png)](https://mermaid.live/edit#pako:eNqVk1FrgzAUhf9KyLMO47aXsBZcV6h0FZntw9A9pOZ2lWmUGAej9L8vadywa0uZQtR7DudeP80O5zUHTPG7ZM0WLR8zgfTRdmtbiLTqodRckPdmxSPDKtHyqgWJ2oblMLAEceylK95M8209KQsQiiLt-wQZNiPi3RB9-k5fimupRsS_vRsEgOBnGs5NwzlIAeVJy-VLECW2K0WSCV5XqNHJA0uo5TCmqB-ADKRFMPFSvQxK8ezVS_VyaSrzjg-um9T5ByjXHfcDWMPBeYYn6XmS8zzJZZ7kh2dygEZRWbQKxH_gkSvwiIVnqKE_kaEWB-j8Y3TkFB25go6coiO_6KzLPk5mQRRNn9N7tFg3rYN8VLV9rvlCOmbcW478pm7GwA6uQFas4PpH3xlLhtUWKsgw1bccNqwrVYYzsddW1qk6-RI5pkp24OCu4UzBU8E0wgrTDStbXQVeqFou7OY57KH9N3Ti7rk)

Now that we have an application, we can see that we had two layers missing. The top two ones: application and transport layers.

The transport layer implements application multiplexing, which allows the same computer to run many services addressed by a port. And with a source port, the two applications can talk with each other on different nodes.

That transport layer can be accessed by our applications via the operating system application programming interface (API). Currently, the most common API format is known as [BSD Sockets](https://docs.freebsd.org/en/books/developers-handbook/sockets/), that were then ported to pretty much every operating system.

Finally, we have the application layer, where we have our applications. They can be anything connected to the internet, including but not limited to: internet banking, streaming, games, websites, etc.

### Applications in ns-3

Applications in ns-3 are slightly different than real applications since we have a simulator instead of an operating system. Let's take a look at the following diagram.

[![](https://mermaid.ink/img/pako:eNqdkz1vAjEMhv-KlZlO7XQDCywMVVFP3W5xE0MjcknqOFSI8t_r61GVgjrATb58PO9rO94bmxyZxhR6rxQtzT2uGfsugn7PhAEw5-Atik_xbjp9ysQaxzWUXRHqP1vf14CSuNHjyihSYLHcPsDLfAkl2Q3JCPv_pmLPlQaYJb-lcmRAYiDmxCMslvszY2crDcyYUAgwXjjBIMAj3p1Cb0pZ2KtJdSfEvY-DpLzRKWKEUyj0q3pq5wbZlqIrkPEv5LIo1zOu7NIiatpc8_AHH7jRUgzZZ06WisbpJ2XA66Qu2tmOz-DZbmcYwquywBewGpM7VjhqYCam10agd_qo98NGZ9RRT51pNHS0whqkM1086FGsktpdtKYRrjQxNTtt4HEGTLNCbdrEkPNq6nEclO95OXwBCO8irg?type=png)](https://mermaid.live/edit#pako:eNqdkz1vAjEMhv-KlZlO7XQDCywMVVFP3W5xE0MjcknqOFSI8t_r61GVgjrATb58PO9rO94bmxyZxhR6rxQtzT2uGfsugn7PhAEw5-Atik_xbjp9ysQaxzWUXRHqP1vf14CSuNHjyihSYLHcPsDLfAkl2Q3JCPv_pmLPlQaYJb-lcmRAYiDmxCMslvszY2crDcyYUAgwXjjBIMAj3p1Cb0pZ2KtJdSfEvY-DpLzRKWKEUyj0q3pq5wbZlqIrkPEv5LIo1zOu7NIiatpc8_AHH7jRUgzZZ06WisbpJ2XA66Qu2tmOz-DZbmcYwquywBewGpM7VjhqYCam10agd_qo98NGZ9RRT51pNHS0whqkM1086FGsktpdtKYRrjQxNTtt4HEGTLNCbdrEkPNq6nEclO95OXwBCO8irg)

As shown, instead of requesting a socket, we just create one from the application.

The other obvious difference is that instead of the application being awoken where it stopped then continue processing, we instead call a function known as a callback to receive and decide what to do with that packet.

Since we already have our topology ready, let's go straight to the application setup. 

First we will need to define two wrappers using C++.

In [2]:
# We need to use this weird trick of defining c++ stuff from Python because
# template forwarding isn't supported by our Cppyy bindings.

ns.cppyy.cppdef("""
    #ifndef AppCallbackAndSend
    #define AppCallbackAndSend
    namespace ns3
    {
        Callback<void,Ptr<Socket> > make_rx_callback(void(*func)(Ptr<Socket>))
        {
            return MakeCallback(func);
        }
        EventImpl* pythonMakeEventSend(void (*f)(Ptr<Socket>, Ptr<Packet>, Address&), Ptr<Socket> socket, Ptr<Packet> packet, Address address)
        {
            return MakeEvent(f, socket, packet, address);
        }
    }
    #endif
""")

True

Next, we can write our own application. In this case, it will be the UdpEchoServer.

[![](https://mermaid.ink/img/pako:eNp1UbtuwzAM_BWCc_oDGrL0sXWpVy-EeI6FypKrR4oiyL9XruAWTVFNEnl3PJ4ubKOCDWe8VQSLByenJMsYqB0f40pSCGJnyrAxaG882jnee4dQ7o7H7TEgnZEMDQiaaRX7itKh2Dmb2A-7E27YL7BwZ-wCJEEp2xlaPciVTFNM75LUhdM_St2UoacOpNQV9ZelP-531s387wX4wAvSIk5bUpetPHKZsWBk066KSaovI4_h2qBSSxw-gmVTUsWB66pS9mDZTOJzq0Jdiem5p__1CddPeIqH4Q?type=png)](https://mermaid.live/edit#pako:eNp1UbtuwzAM_BWCc_oDGrL0sXWpVy-EeI6FypKrR4oiyL9XruAWTVFNEnl3PJ4ubKOCDWe8VQSLByenJMsYqB0f40pSCGJnyrAxaG882jnee4dQ7o7H7TEgnZEMDQiaaRX7itKh2Dmb2A-7E27YL7BwZ-wCJEEp2xlaPciVTFNM75LUhdM_St2UoacOpNQV9ZelP-531s387wX4wAvSIk5bUpetPHKZsWBk066KSaovI4_h2qBSSxw-gmVTUsWB66pS9mDZTOJzq0Jdiem5p__1CddPeIqH4Q)

In [3]:
import sys
class EchoServer(ns.applications.Application):
    LOGGING = True
    ECHO_PORT = 1234 # Application default listening port
    socketToInstanceDict = {}

    def __init__(self, node: ns.Node, port=ECHO_PORT):
        super().__init__() # Calls ns.Application constructor
        self.__python_owns__ = False  # Allow C++ to destroy this object when Simulator::Destroy gets called
        self.port = port # Server listening port
        
        # Create an UDP socket
        self.m_socket = ns.network.Socket.CreateSocket(node,
                                                       ns.core.TypeId.LookupByName("ns3::UdpSocketFactory"))
        # Bind that socket to a specific port and IP addresses to listen
        self.m_socket.Bind(ns.network.InetSocketAddress(ns.network.Ipv4Address.GetAny(), self.port).ConvertTo())
        
        # Creates the callback to handle packet reception using a static function of the EchoServerClass
        # (This is a workaround for the application to work from Python)
        self.m_socket.SetRecvCallback(ns.make_rx_callback(EchoServer._Receive))
        
        # Registers the socket as a dictionary key and associate it to this application instance
        EchoServer.socketToInstanceDict[self.m_socket] = self

    def __del__(self):
        # Remove the instance entry on the class dictioanry when the object is destroyed
        del EchoServer.socketToInstanceDict[self.m_socket]

    def Send(self, packet: ns.Packet, address: ns.Address) -> None:
        # Function that sends the packet to a target address.
        self.m_socket.SendTo(packet, 0, address)
        if EchoServer.LOGGING:
            inetAddress = ns.InetSocketAddress.ConvertFrom(address)
            print("At time +{s}s server sent {b} bytes from {ip} port {port}"
                  .format(s=ns.Simulator.Now().GetSeconds(),
                          b=packet.__deref__().GetSize(),
                          ip=inetAddress.GetIpv4(),
                          port=inetAddress.GetPort()),
                  file=sys.stderr,
                  flush=True)

    def Receive(self):
        # Function that receives a packet and the address of the sender
        address = ns.Address()
        packet = self.m_socket.RecvFrom(address)
        if EchoServer.LOGGING:
            inetAddress = ns.InetSocketAddress.ConvertFrom(address)
            print("At time +{s}s server received {b} bytes from {ip} port {port}"
                  .format(s=ns.Simulator.Now().GetSeconds(),
                          b=packet.__deref__().GetSize(),
                          ip=inetAddress.GetIpv4(),
                          port=inetAddress.GetPort()),
                  file=sys.stderr,
                  flush=True)
        # Schedule the Send function to send the packet back to its sender after 1 second
        event = ns.pythonMakeEventSend(EchoServer._Send, self.m_socket, packet, address)
        ns.Simulator.Schedule(ns.Seconds(1), event)

    @staticmethod
    def _Send(socket: ns.Socket, packet: ns.Packet, address: ns.Address):
        # Static function that identifies the EchoServer instance that is
        # supposed to send a packet and calls it to send
        instance = EchoServer.socketToInstanceDict[socket]
        instance.Send(packet, address)
        pass

    @staticmethod
    def _Receive(socket: ns.Socket) -> None:
        # Static function that identifies the EchoServer instance that is
        # supposed to receive a packet and calls it to receive
        instance = EchoServer.socketToInstanceDict[socket]
        instance.Receive()
        pass

After the application is ready, we can use it in our topology.

In [4]:
echoServer = EchoServer(nodes.Get(1))
nodes.Get(1).AddApplication(echoServer)

serverApps = ns.ApplicationContainer()
serverApps.Add(echoServer)
serverApps.Start(ns.core.Seconds(1.0))
serverApps.Stop(ns.core.Seconds(10.0))

address = interfaces.GetAddress(1).ConvertTo()

ns.LogComponentEnable("UdpEchoClientApplication", ns.core.LOG_LEVEL_INFO)
echoClient = ns.applications.UdpEchoClientHelper(address, EchoServer.ECHO_PORT)
echoClient.SetAttribute("MaxPackets", ns.core.UintegerValue(10))
echoClient.SetAttribute("Interval", ns.core.TimeValue(ns.core.Seconds(1.0)))
echoClient.SetAttribute("PacketSize", ns.core.UintegerValue(101))

clientApps = echoClient.Install(nodes.Get(0))
clientApps.Start(ns.core.Seconds(2.0))
clientApps.Stop(ns.core.Seconds(10.0))

ns.Simulator.Run()
ns.Simulator.Destroy()
ns.FatalImpl.FlushStreams() # Make sure the pcaps are written

At time +2s client sent 101 bytes to 10.1.1.2 port 1234
At time +2.0022096s server received 101 bytes from 10.1.1.1 port 49153
At time +3s client sent 101 bytes to At time +3.0022096s server sent 101 bytes from 10.1.1.1 port 49153
10.1.1.2 port 1234
At time +3.0022096s server received 101 bytes from 10.1.1.1 port 49153
At time +4.0022096s server sent 101 bytes from 10.1.1.1 port 49153At time +3.00442s client received 101 bytes from 10.1.1.2 port 1234
At time +4s client sent 101 bytes to 10.1.1.2 port 1234

At time +4.0022096s server received 101 bytes from 10.1.1.1 port 49153
At time +5.0022096s server sent 101 bytes from 10.1.1.1 port 49153
At time +4.00442s client received 101 bytes from 10.1.1At time +5.0022096s server received 101 bytes from 10.1.1.1 port 49153
.2 port 1234
At time +5s client sent 101 bytes to 10.1.1.2 port 1234
At time +6.0022096s server sent 101 bytes from 10.1.1.1 port 49153
At time +5.00442s client received 101 bytes from 10.1.1.2 port 1234
At time +6s client s

As we can se printed above, our server application works. Due to how the bindings handles output, the messages may be out of order, but as long as the execution time of the examples worked, everything should be working just fine.

We can also see our Pcap to double check that.

In [8]:
def print_packets_with_tshark(packet_i):
    import pyshark
    cap = pyshark.FileCapture('application-0-0.pcap')
    i = 0
    for packet in cap:
        if packet_i == i:
            print(packet)
            break
        i += 1
    cap.close()
    
# Jupyter shenanigans with Pyshark asyncio code
import concurrent.futures
def exec_async(func, *args, **kwargs):
    with concurrent.futures.ThreadPoolExecutor(max_workers=1) as executor:
        future = executor.submit(func, *args, **kwargs)
    return future.result()
result = exec_async(print_packets_with_tshark, packet_i=1)

Packet (Length: 131)
Layer PPP
:	Protocol: Internet Protocol version 4 (0x0021)
Layer IP
:	0100 .... = Version: 4
	.... 0101 = Header Length: 20 bytes (5)
	Differentiated Services Field: 0x00 (DSCP: CS0, ECN: Not-ECT)
	0000 00.. = Differentiated Services Codepoint: Default (0)
	.... ..00 = Explicit Congestion Notification: Not ECN-Capable Transport (0)
	Total Length: 129
	Identification: 0x0000 (0)
	000. .... = Flags: 0x0
	0... .... = Reserved bit: Not set
	.0.. .... = Don't fragment: Not set
	..0. .... = More fragments: Not set
	...0 0000 0000 0000 = Fragment Offset: 0
	Time to Live: 64
	Protocol: UDP (17)
	Header Checksum: 0x0000 [validation disabled]
	Header checksum status: Unverified
	Source Address: 10.1.1.1
	Destination Address: 10.1.1.2
Layer UDP
:	Source Port: 49153
	Destination Port: 1234
	Length: 109
	Checksum: 0x0000 [zero-value ignored]
	Checksum Status: Not present
	Stream index: 0
	Timestamps
	Time since first frame: 0.000000000 seconds
	Time since previous frame: 0.0000

We can see above a packet from the EchoClient, which is an IPv4 packet with UDP transport protocol. 

Source address refers to the Node 0, while the target address refers to the Node 1. 

The payload has 101 bytes, as configured in `echoClient.SetAttribute("PacketSize", ns.core.UintegerValue(101))`.

In [10]:
result = exec_async(print_packets_with_tshark, packet_i=2)

Packet (Length: 131)
Layer PPP
:	Protocol: Internet Protocol version 4 (0x0021)
Layer IP
:	0100 .... = Version: 4
	.... 0101 = Header Length: 20 bytes (5)
	Differentiated Services Field: 0x00 (DSCP: CS0, ECN: Not-ECT)
	0000 00.. = Differentiated Services Codepoint: Default (0)
	.... ..00 = Explicit Congestion Notification: Not ECN-Capable Transport (0)
	Total Length: 129
	Identification: 0x0000 (0)
	000. .... = Flags: 0x0
	0... .... = Reserved bit: Not set
	.0.. .... = Don't fragment: Not set
	..0. .... = More fragments: Not set
	...0 0000 0000 0000 = Fragment Offset: 0
	Time to Live: 64
	Protocol: UDP (17)
	Header Checksum: 0x0000 [validation disabled]
	Header checksum status: Unverified
	Source Address: 10.1.1.2
	Destination Address: 10.1.1.1
Layer UDP
:	Source Port: 1234
	Destination Port: 49153
	Length: 109
	Checksum: 0x0000 [zero-value ignored]
	Checksum Status: Not present
	Stream index: 0
	Timestamps
	Time since first frame: 1.004419000 seconds
	Time since previous frame: 0.0044

We can see above a packet from the EchoServer. 

Same everything, but with a delay of 1 second (`Time since first frame: 1.004419000 seconds`), since our EchoServer sends the message back to the EchoClient a second later.

Source address refers to the Node 1, while the target address refers to the Node 0.